In [1]:
from multitask_dataset import SingerMultiTaskDataset
from torch.utils.data import DataLoader
from torch.optim import Adam
from models import HuBERTMultiHead
import pandas as pd

import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

from torchview import draw_graph
import torchvision
# from torchviz import make_dot
from torchsummary import summary

# import torch
# torch.autograd.detect_anomaly(True)

/home/maximos/miniconda3/envs/torchaudio/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_audio_folder = '/media/maindisk/maximos/data/melos_singers/Rebetika_vowels/train/'
csv_path = '/media/maindisk/maximos/data/melos_singers/features/multitask_targets.csv'

In [3]:
feats = pd.read_csv(csv_path, delimiter=',')
features_list = list(feats.columns)
print(features_list)
del(features_list[:2])
print(features_list)

['Unnamed: 0', 'names', 'singer_id', 'Pitch', 'SpectralCentroid', 'SpectralSpread', 'SpectralSkewness', 'SpectralKurtosis', 'SpectralFlatness', 'SpectralCrest', 'SpectralSlope', 'SpectralDecrease', 'SpectralRollOff', 'SpectralVariation', 'SpectralFlux', 'HarmonicSpectralDeviation', 'Tristimulus_1', 'Tristimulus_2', 'Tristimulus_3', 'HarmonicOddToEvenRatio', 'Inharmonicity', 'HarmonicEnergy', 'NoiseEnergy', 'Noisiness', 'HarmonicToNoiseEnergy', 'PartialsToNoiseEnergy', 'F1_Hz', 'F2_Hz', 'F3_HZ', 'F4_Hz', 'Rate', 'Depth', 'Regularity']
['singer_id', 'Pitch', 'SpectralCentroid', 'SpectralSpread', 'SpectralSkewness', 'SpectralKurtosis', 'SpectralFlatness', 'SpectralCrest', 'SpectralSlope', 'SpectralDecrease', 'SpectralRollOff', 'SpectralVariation', 'SpectralFlux', 'HarmonicSpectralDeviation', 'Tristimulus_1', 'Tristimulus_2', 'Tristimulus_3', 'HarmonicOddToEvenRatio', 'Inharmonicity', 'HarmonicEnergy', 'NoiseEnergy', 'Noisiness', 'HarmonicToNoiseEnergy', 'PartialsToNoiseEnergy', 'F1_Hz', '

In [4]:
feats['singer_id']

0      0
1      0
2      0
3      0
4      0
      ..
380    6
381    6
382    6
383    6
384    6
Name: singer_id, Length: 385, dtype: int64

In [5]:
task_labels_num_out = {}
for i in range(1, len(features_list)-3, 1):
    task_labels_num_out[features_list[i]] = 1
# add singer identification
task_labels_num_out['singer_id'] = feats['singer_id'].max()+1 # accounting for zero
print(task_labels_num_out)

{'Pitch': 1, 'SpectralCentroid': 1, 'SpectralSpread': 1, 'SpectralSkewness': 1, 'SpectralKurtosis': 1, 'SpectralFlatness': 1, 'SpectralCrest': 1, 'SpectralSlope': 1, 'SpectralDecrease': 1, 'SpectralRollOff': 1, 'SpectralVariation': 1, 'SpectralFlux': 1, 'HarmonicSpectralDeviation': 1, 'Tristimulus_1': 1, 'Tristimulus_2': 1, 'Tristimulus_3': 1, 'HarmonicOddToEvenRatio': 1, 'Inharmonicity': 1, 'HarmonicEnergy': 1, 'NoiseEnergy': 1, 'Noisiness': 1, 'HarmonicToNoiseEnergy': 1, 'PartialsToNoiseEnergy': 1, 'F1_Hz': 1, 'F2_Hz': 1, 'F3_HZ': 1, 'F4_Hz': 1, 'singer_id': 7}


In [6]:
model = HuBERTMultiHead(task_labels_num_out=task_labels_num_out)

You are using a model of type hubert to instantiate a model of type wav2vec2. This is not supported for all configurations of models and can yield errors.
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
training_data = SingerMultiTaskDataset(train_audio_folder, csv_path)

In [8]:
print(len(training_data))

336


In [9]:
print(training_data[335])

335 tsaousakis_i_7 {'index': [355], 'columns': ['Unnamed: 0', 'names', 'singer_id', 'Pitch', 'SpectralCentroid', 'SpectralSpread', 'SpectralSkewness', 'SpectralKurtosis', 'SpectralFlatness', 'SpectralCrest', 'SpectralSlope', 'SpectralDecrease', 'SpectralRollOff', 'SpectralVariation', 'SpectralFlux', 'HarmonicSpectralDeviation', 'Tristimulus_1', 'Tristimulus_2', 'Tristimulus_3', 'HarmonicOddToEvenRatio', 'Inharmonicity', 'HarmonicEnergy', 'NoiseEnergy', 'Noisiness', 'HarmonicToNoiseEnergy', 'PartialsToNoiseEnergy', 'F1_Hz', 'F2_Hz', 'F3_HZ', 'F4_Hz', 'Rate', 'Depth', 'Regularity'], 'data': [[355, 'tsaousakis_i_7', 6, 0.17002993509861025, 0.2869796497012348, 0.46253597354674036, 0.3055921244762052, 0.03667717739558078, 7.931476365545892e-09, 0.5020039152143873, 0.6035755424096567, 0.041501375576359925, 0.33524749624698796, 0.8079429365707972, 0.14093257873051493, 0.3126923231645448, 0.1980443400802777, 0.6489481209528397, 0.22971928529167335, 0.0011820919656826887, 0.2565523860585552, 0.

In [10]:
dataloader = DataLoader(training_data, batch_size=4, shuffle=True, collate_fn=model.collate_fn)

In [11]:
print(dataloader.__dict__)

{'dataset': <multitask_dataset.SingerMultiTaskDataset object at 0x74f0e818c6e0>, 'num_workers': 0, 'prefetch_factor': None, 'pin_memory': False, 'pin_memory_device': '', 'timeout': 0, 'worker_init_fn': None, '_DataLoader__multiprocessing_context': None, '_dataset_kind': 0, 'batch_size': 4, 'drop_last': False, 'sampler': <torch.utils.data.sampler.RandomSampler object at 0x74eff5074c50>, 'batch_sampler': <torch.utils.data.sampler.BatchSampler object at 0x74eff4f54da0>, 'generator': None, 'collate_fn': <bound method HuBERTMultiHead.collate_fn of HuBERTMultiHead(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
      

In [33]:
b = next(iter(dataloader))

92 kazantzidis_w_12 {'index': [191], 'columns': ['Unnamed: 0', 'names', 'singer_id', 'Pitch', 'SpectralCentroid', 'SpectralSpread', 'SpectralSkewness', 'SpectralKurtosis', 'SpectralFlatness', 'SpectralCrest', 'SpectralSlope', 'SpectralDecrease', 'SpectralRollOff', 'SpectralVariation', 'SpectralFlux', 'HarmonicSpectralDeviation', 'Tristimulus_1', 'Tristimulus_2', 'Tristimulus_3', 'HarmonicOddToEvenRatio', 'Inharmonicity', 'HarmonicEnergy', 'NoiseEnergy', 'Noisiness', 'HarmonicToNoiseEnergy', 'PartialsToNoiseEnergy', 'F1_Hz', 'F2_Hz', 'F3_HZ', 'F4_Hz', 'Rate', 'Depth', 'Regularity'], 'data': [[191, 'kazantzidis_w_12', 2, 0.22721392462383172, 0.46643158554604464, 0.7252600561514136, 0.6189324576743124, 0.3619349651759216, 1.3994632094969663e-06, 0.16996052400048403, 0.48377683031172647, 0.03305770201650879, 0.32103312570969766, 0.8066427090996909, 0.06439847388939765, 0.07007052839574299, 0.12063898775620663, 0.5247342374585208, 0.3798223227161126, 0.02187298869076925, 0.33262817765553615

In [13]:
print(training_data.names[223])
print(training_data.feats['names'].str.contains('kazantzidisold').any())

kazantzidisold_w_10
True


In [14]:
print(len(b[0]['input_values']))
print(len(b[0]['attention_mask']))
print(len(b[1]))
print(b[1])

4
4
33
{'Unnamed: 0': [359, 186, 260, 282], 'names': ['tsaousakis_u_10', 'kazantzidis_u_8', 'keti_grei_w_5', 'ninou_i_5'], 'singer_id': [6, 2, 4, 5], 'Pitch': [0.15682407717194807, 0.31650664171835347, 0.59542295165445, 0.4665907252114494], 'SpectralCentroid': [0.12098630611789565, 0.30400680387124124, 0.37980433720761564, 0.631807958886131], 'SpectralSpread': [0.2869807018981558, 0.40138912463835696, 0.4729555620903711, 0.5776180416874821], 'SpectralSkewness': [0.6111897837510597, 0.5586589300090187, 0.7979034877970709, 0.1019738582291091], 'SpectralKurtosis': [0.31560845841580126, 0.27798707340641127, 0.5884459337286994, 0.0], 'SpectralFlatness': [3.8452336010885656e-07, 4.4141003206019717e-08, 2.0904907946220324e-08, 0.0004789809847429196], 'SpectralCrest': [0.4920409522948828, 0.29177882280174444, 0.5140478468783976, 0.30324303498948024], 'SpectralSlope': [0.5239149966779877, 0.5300203240297618, 0.5375386797380108, 0.49796168891063264], 'SpectralDecrease': [0.10905891087464421, 0.0

In [15]:
# model.train()
# model.eval()
for p in model.hubert.parameters():
    p.requires_grad = False
train_params = []
for k in model.projectors.keys():
    train_params.extend( list(model.projectors[k].parameters()) )
    train_params.extend( list(model.classifiers[k].parameters()) )
    print(k)
    model.projectors[k].requires_grad = True
    model.classifiers[k].requires_grad = True
# # model.train(False)
# print(train_params)
# optimizer = Adam( model.classifiers['Pitch'].parameters() )
optimizer = Adam( model.parameters(), lr=0.001 )

Pitch
SpectralCentroid
SpectralSpread
SpectralSkewness
SpectralKurtosis
SpectralFlatness
SpectralCrest
SpectralSlope
SpectralDecrease
SpectralRollOff
SpectralVariation
SpectralFlux
HarmonicSpectralDeviation
Tristimulus_1
Tristimulus_2
Tristimulus_3
HarmonicOddToEvenRatio
Inharmonicity
HarmonicEnergy
NoiseEnergy
Noisiness
HarmonicToNoiseEnergy
PartialsToNoiseEnergy
F1_Hz
F2_Hz
F3_HZ
F4_Hz
singer_id


In [16]:
# for p in model.parameters():
#     print(p)
print(model.parameters())

<generator object Module.parameters at 0x74eff506d620>


In [17]:
print(optimizer)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


In [18]:
print(model.hubert.encoder.layers[0].attention.k_proj.weight.grad)

None


In [19]:
b[0]['attention_mask']

[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [20]:
print(b[1]['singer_id'])

[6, 2, 4, 5]


In [21]:
y = model(
    audio_normalized=b[0]['input_values'],
    attention_mask=b[0]['attention_mask'],
    labels=b[1],
    output_attentions=False,
    output_hidden_states=False,
    return_dict=True
)

In [22]:
print(y)

SequenceClassifierOutput(loss=tensor(4.4003, device='cuda:0', grad_fn=<AddBackward0>), logits={'Pitch': [tensor([[0.5026],
        [0.5021],
        [0.5074],
        [0.5090]], device='cuda:0', grad_fn=<SigmoidBackward0>)], 'SpectralCentroid': [tensor([[0.5040],
        [0.5051],
        [0.5065],
        [0.4941]], device='cuda:0', grad_fn=<SigmoidBackward0>)], 'SpectralSpread': [tensor([[0.4998],
        [0.5041],
        [0.4987],
        [0.5022]], device='cuda:0', grad_fn=<SigmoidBackward0>)], 'SpectralSkewness': [tensor([[0.4983],
        [0.4994],
        [0.5019],
        [0.5089]], device='cuda:0', grad_fn=<SigmoidBackward0>)], 'SpectralKurtosis': [tensor([[0.5050],
        [0.5067],
        [0.5079],
        [0.5010]], device='cuda:0', grad_fn=<SigmoidBackward0>)], 'SpectralFlatness': [tensor([[0.5159],
        [0.5152],
        [0.5132],
        [0.5187]], device='cuda:0', grad_fn=<SigmoidBackward0>)], 'SpectralCrest': [tensor([[0.5070],
        [0.5138],
        [0.5067],


In [23]:
print(y['logits'].keys())
for k in y['logits'].keys():
    print(y['logits'][k])

dict_keys(['Pitch', 'SpectralCentroid', 'SpectralSpread', 'SpectralSkewness', 'SpectralKurtosis', 'SpectralFlatness', 'SpectralCrest', 'SpectralSlope', 'SpectralDecrease', 'SpectralRollOff', 'SpectralVariation', 'SpectralFlux', 'HarmonicSpectralDeviation', 'Tristimulus_1', 'Tristimulus_2', 'Tristimulus_3', 'HarmonicOddToEvenRatio', 'Inharmonicity', 'HarmonicEnergy', 'NoiseEnergy', 'Noisiness', 'HarmonicToNoiseEnergy', 'PartialsToNoiseEnergy', 'F1_Hz', 'F2_Hz', 'F3_HZ', 'F4_Hz', 'singer_id'])
[tensor([[0.5026],
        [0.5021],
        [0.5074],
        [0.5090]], device='cuda:0', grad_fn=<SigmoidBackward0>)]
[tensor([[0.5040],
        [0.5051],
        [0.5065],
        [0.4941]], device='cuda:0', grad_fn=<SigmoidBackward0>)]
[tensor([[0.4998],
        [0.5041],
        [0.4987],
        [0.5022]], device='cuda:0', grad_fn=<SigmoidBackward0>)]
[tensor([[0.4983],
        [0.4994],
        [0.5019],
        [0.5089]], device='cuda:0', grad_fn=<SigmoidBackward0>)]
[tensor([[0.5050],
    

In [24]:
optimizer.zero_grad()
y.loss.backward()# retain_graph=True
optimizer.step()

In [25]:
print(model)

HuBERTMultiHead(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=768, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): Wav2Vec2Encoder(
    (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(


In [26]:
print(model.classifiers['Pitch'].weight.grad)
# print(model.projectors['F4_Hz'].weight)

tensor([[ 2.0024e-03,  1.3027e-04,  1.3854e-03, -6.1435e-04, -1.1231e-05,
         -3.5320e-05,  5.1061e-03,  3.3670e-03, -3.4278e-05,  4.5540e-03,
          1.9442e-04,  7.3356e-04,  8.1223e-04,  9.4148e-04, -4.4047e-04,
          1.2218e-03,  2.2363e-03, -6.7758e-05,  1.1447e-03, -3.1392e-04,
          6.2463e-03,  2.3859e-03,  5.8369e-03,  1.4306e-03, -2.1870e-04,
         -1.7073e-05,  2.6793e-03,  1.1445e-04,  2.2785e-03,  7.5386e-05,
         -7.9995e-05,  6.4741e-04,  5.8690e-04, -1.4844e-05, -1.6117e-06,
         -1.6052e-04, -4.8105e-05,  4.0965e-04,  2.5742e-03,  1.4968e-03,
          2.6679e-03, -3.3105e-05,  2.3473e-03,  8.4192e-04,  1.1323e-03,
          2.3459e-03, -2.2027e-04,  3.5927e-04,  3.5580e-03,  6.1592e-03,
          9.5735e-04,  1.6343e-03,  4.1146e-03,  5.3934e-03,  5.5743e-05,
         -6.0250e-05,  4.9292e-04,  3.2822e-05, -2.7516e-05,  6.5787e-05,
          1.2373e-03,  3.9049e-04,  1.6801e-03,  1.7747e-03, -2.4827e-05,
          2.0134e-03, -2.5912e-05, -1.

In [27]:
print(model.hubert.encoder.layers[11].final_layer_norm.weight.grad)
print(model.hubert.encoder.layers[11].final_layer_norm.weight)

None
Parameter containing:
tensor([0.2295, 0.3408, 0.2241, 0.2180, 0.3718, 0.2739, 0.4177, 0.7271, 0.2072,
        0.2150, 0.2554, 0.2473, 0.2111, 0.7178, 0.1851, 0.2050, 0.4814, 0.2057,
        0.2379, 0.1952, 0.5957, 0.3611, 0.2522, 0.5278, 0.1882, 1.0547, 0.2152,
        0.4490, 0.4299, 0.2314, 0.2019, 0.2072, 0.2798, 0.2083, 0.4707, 0.2754,
        0.2136, 0.6519, 0.2372, 0.2583, 0.2527, 0.2141, 0.2001, 0.3301, 0.2196,
        0.2537, 0.2581, 0.2908, 0.2773, 0.2332, 0.1733, 0.2766, 0.2544, 0.2703,
        0.2329, 0.2493, 0.2891, 0.2959, 0.2852, 0.6724, 0.2040, 0.2484, 0.2087,
        0.3169, 0.2595, 0.1816, 0.2347, 0.7002, 0.2201, 0.2742, 0.2114, 0.2729,
        0.6841, 0.6235, 0.4590, 0.1656, 0.2288, 0.4587, 0.3438, 0.2225, 0.4719,
        0.2688, 0.2349, 0.2260, 0.2186, 0.2642, 0.2301, 0.3638, 0.2355, 0.4534,
        0.1997, 0.1818, 0.1923, 0.2277, 0.7339, 0.2678, 0.2124, 0.2478, 0.2356,
        0.3901, 0.2098, 0.2205, 0.2212, 0.6611, 0.2272, 0.2209, 0.2822, 0.2277,
        0.570

In [28]:
print(len(model.hubert.encoder.layers))

12


In [29]:
print(model.hubert.encoder.layers[11].feed_forward.output_dense.weight.grad)
print(model.hubert.encoder.layers[11].feed_forward.output_dense.weight)
# print(model.hubert.encoder.layers[0].attention.k_proj.weight.grad)

None
Parameter containing:
tensor([[-0.0309, -0.0560, -0.0944,  ..., -0.0551,  0.0054, -0.1141],
        [-0.0452,  0.0558, -0.0271,  ..., -0.0301, -0.0216,  0.0928],
        [ 0.0855, -0.0440,  0.1083,  ..., -0.0047,  0.0585, -0.1077],
        ...,
        [-0.0461, -0.0809,  0.0314,  ...,  0.0263,  0.0117,  0.0425],
        [-0.0258, -0.0419, -0.0305,  ..., -0.0015, -0.0509, -0.1317],
        [-0.0070, -0.0095,  0.0326,  ...,  0.0144,  0.0219, -0.0197]],
       device='cuda:0')


In [30]:
for i in range(50):
    y = model(
        audio_normalized=b[0]['input_values'],
        attention_mask=b[0]['attention_mask'],
        labels=b[1],
        output_attentions=False,
        output_hidden_states=False,
        return_dict=True
    )
    print(y.loss)
    # print(model.classifiers['F4_Hz'].weight)
    optimizer.zero_grad()
    y.loss.backward()
    optimizer.step()

tensor(3.7831, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.2644, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7722, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.4376, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.2265, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.1175, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.0295, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.9216, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.8184, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7184, device='cuda:0', grad_fn=<AddBackward0>)


tensor(1.6555, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.5979, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.5495, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.4796, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.4130, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.3952, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.3545, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.3241, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2803, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2705, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2499, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2371, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2325, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2129, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2202, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2086, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2011, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2069, device='cuda:0', grad_fn=<AddBack

In [31]:
print(y['logits']['singer_id'][0].argmax(dim=1, keepdim=True).squeeze())

tensor([6, 2, 4, 5], device='cuda:0')
